### Environment Setup

In [94]:
%pip install -q -r requirements.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [95]:
from dotenv import load_dotenv
_ = load_dotenv()

In the Basics notebook, we created templates and invoked them and finally feed the result to the LLM. We can make this easier by using Langchain expression language which is very similar to the piping in the Linux shells.

In [96]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [97]:
out = llm.invoke("Hello, who are you?")
print(out.content)

Hello! I’m an AI language model created by OpenAI. I'm here to assist you with information, answer questions, and engage in conversation. How can I help you today?


### Tools

Tools can be passed to chat models that support tool calling allowing the model to request the execution of a specific function with specific inputs if it decides to use it. A tool can be a very simple function and also complex functions that can be used to perform complex operations.  
`@tool` decorator is used to create tools that can be used in the model call.

##### Defining

In [98]:
from langchain_core.tools import tool

@tool
def multiply(a: int, b: int) -> int:
   """Multiply two numbers."""
   return a * b

In [99]:
print(f"{multiply.name=}")
print(f"{multiply.description=}")
print(f"{multiply.args=}")


multiply.name='multiply'
multiply.description='Multiply two numbers.'
multiply.args={'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [100]:
# self test
multiply.invoke({'a':2, 'b':3})

6

A more complex tool can be defined as the following

In [101]:
%pip install -q yfinance

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [102]:
import yfinance as yf

@tool
def get_stock_price(symbol):
    """Get the latest price of stock 'symbol'"""
    ticker = yf.Ticker(symbol)
    todays_data = ticker.history(period='1d')
    return float(todays_data.iloc[0]['Close'])

In [103]:
get_stock_price.invoke({'symbol':"AAPL"})

238.9062957763672

In [104]:
get_stock_price.invoke("AAPL")

238.9062957763672

#### Binding

We can bind the tools to the model in three ways:

1. In constructor with kwargs
2. via invoke
3. After construction with bind_tools

##### Constructor with kwargs (model specific)

We can pass the tools to the underlying LLM model, but the tool should have exactly the same structure as LLM expects. Therefore we need to wrap the tool using convert_to_openai_tool.

In [105]:
# Instantiate the model with the tool
from langchain_core.utils.function_calling import convert_to_openai_tool
openai_tools = [convert_to_openai_tool(get_stock_price)]

llm_with_default_tools = ChatOpenAI(model_kwargs={'tools':openai_tools})

In [106]:
llm_with_default_tools.invoke("What is the price of GOOGLE stock?")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_lwNilinnaYg06ifa2I1bomew', 'function': {'arguments': '{"symbol":"GOOGLE"}', 'name': 'get_stock_price'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 56, 'total_tokens': 73, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d99acb6c-533c-4386-b295-271fa2b22e29-0', tool_calls=[{'name': 'get_stock_price', 'args': {'symbol': 'GOOGLE'}, 'id': 'call_lwNilinnaYg06ifa2I1bomew', 'type': 'tool_call'}], usage_metadata={'input_tokens': 56, 'output_tokens': 17, 'total_tokens': 73, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

##### via invoke (model specific)

In [107]:
llm.invoke("What is the price of GOOGLE stock?", tools=openai_tools)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_h6GAswTpI9ANErcby27vB8PE', 'function': {'arguments': '{"symbol":"GOOGL"}', 'name': 'get_stock_price'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 55, 'total_tokens': 73, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-ac35ac60-ed99-4f76-aaf0-626fa0209303-0', tool_calls=[{'name': 'get_stock_price', 'args': {'symbol': 'GOOGL'}, 'id': 'call_h6GAswTpI9ANErcby27vB8PE', 'type': 'tool_call'}], usage_metadata={'input_tokens': 55, 'output_tokens': 18, 'total_tokens': 73, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'rea

##### via langchain bind_tools

Easiest, model independent, and most flexible way is to use bind_tools

In [108]:
llm_with_binded_tools = llm.bind_tools([get_stock_price])

In [109]:
out = llm_with_binded_tools.invoke("What is the price of GOOGLE stock?")
out

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_JF0f5rQTGm1708zMZs1VYvEf', 'function': {'arguments': '{"symbol":"GOOGL"}', 'name': 'get_stock_price'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 55, 'total_tokens': 73, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-3441d54a-0b71-44e7-8dfa-8ef22d89d157-0', tool_calls=[{'name': 'get_stock_price', 'args': {'symbol': 'GOOGL'}, 'id': 'call_JF0f5rQTGm1708zMZs1VYvEf', 'type': 'tool_call'}], usage_metadata={'input_tokens': 55, 'output_tokens': 18, 'total_tokens': 73, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'rea

As seen model succesfully returned the hint to call the tool with the correct parameters.

In [110]:
out.tool_calls

[{'name': 'get_stock_price',
  'args': {'symbol': 'GOOGL'},
  'id': 'call_JF0f5rQTGm1708zMZs1VYvEf',
  'type': 'tool_call'}]

If we asked a question that the model doesn't decide to use a tool, the tool_calls will be empty.

In [111]:
out = llm_with_binded_tools.invoke("What is wheather in New York?")

In [112]:
out

AIMessage(content="I currently don't have access to real-time weather data. You can check a reliable weather website or app for the latest weather updates in New York.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 55, 'total_tokens': 86, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'stop', 'logprobs': None}, id='run-9cef2dc7-698a-4bf5-8c7d-25824fa3bf72-0', usage_metadata={'input_tokens': 55, 'output_tokens': 31, 'total_tokens': 86, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

#### Executing

Tool calling is not useful without executing the tool. Simply the args in the result of tool_calls of model response can be passed to the tool to get the result.

In [113]:
out = llm_with_binded_tools.invoke("What is the price of GOOGLE stock?")
out

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_2S9hafQXn8fKsTBRedVyaTrb', 'function': {'arguments': '{"symbol":"GOOGL"}', 'name': 'get_stock_price'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 55, 'total_tokens': 73, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-a3b19e12-49a5-4701-b5d8-e228b050cb1e-0', tool_calls=[{'name': 'get_stock_price', 'args': {'symbol': 'GOOGL'}, 'id': 'call_2S9hafQXn8fKsTBRedVyaTrb', 'type': 'tool_call'}], usage_metadata={'input_tokens': 55, 'output_tokens': 18, 'total_tokens': 73, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'rea

In [114]:
get_stock_price.invoke(out.tool_calls[0]['args'])

195.47500610351562

#### End2End

In a real-world scenario, we can use the tools to perform complex operations and feed the result to the model and model will responed with human friendly message considering tool result.  
This can be accomplished manually, builtin-agents, chains and graphs. Graphs will be introduced in the next notebooks.

##### Very Manual

In [115]:
# define the toolset
tools = [get_stock_price, multiply]

In [116]:
# start conversation
from langchain_core.messages import HumanMessage, ToolMessage

messages = [HumanMessage("Hello, what is the last price of Google stock?")]
out = llm_with_binded_tools.invoke(messages)
# add returned AI message to the conversation
messages.append(out)
out

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Nnz8xI40Abjd5pODW3UaMy7z', 'function': {'arguments': '{"symbol":"GOOGL"}', 'name': 'get_stock_price'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 58, 'total_tokens': 76, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-617b1158-0d02-4eff-8307-080c81351878-0', tool_calls=[{'name': 'get_stock_price', 'args': {'symbol': 'GOOGL'}, 'id': 'call_Nnz8xI40Abjd5pODW3UaMy7z', 'type': 'tool_call'}], usage_metadata={'input_tokens': 58, 'output_tokens': 18, 'total_tokens': 76, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'rea

In [117]:
# get call details
tc = out.tool_calls[0]
tc

{'name': 'get_stock_price',
 'args': {'symbol': 'GOOGL'},
 'id': 'call_Nnz8xI40Abjd5pODW3UaMy7z',
 'type': 'tool_call'}

In [118]:
# find relevant langchain wrapped tool and invoke it. tc can be passed as is so that it returns a ToolMessage object just as we need.
# otherwise if you pass args, you'll just get the return value of the tool.
tool_call_result = {tool.name: tool for tool in tools}[tc['name']].invoke(tc)
tool_call_result

ToolMessage(content='195.47500610351562', name='get_stock_price', tool_call_id='call_Nnz8xI40Abjd5pODW3UaMy7z')

In [119]:
# add the tool result to the conversation and call LLM again
messages.append(tool_call_result)
messages


[HumanMessage(content='Hello, what is the last price of Google stock?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Nnz8xI40Abjd5pODW3UaMy7z', 'function': {'arguments': '{"symbol":"GOOGL"}', 'name': 'get_stock_price'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 58, 'total_tokens': 76, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-617b1158-0d02-4eff-8307-080c81351878-0', tool_calls=[{'name': 'get_stock_price', 'args': {'symbol': 'GOOGL'}, 'id': 'call_Nnz8xI40Abjd5pODW3UaMy7z', 'type': 'tool_call'}], usage_metadata={'input_tokens': 58, 'output_tokens': 1

In [120]:
llm_with_binded_tools.invoke(messages)

AIMessage(content='The last price of Google stock (GOOGL) is $195.48.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 91, 'total_tokens': 110, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'stop', 'logprobs': None}, id='run-75312e84-edca-45bf-889e-d583fe099e97-0', usage_metadata={'input_tokens': 91, 'output_tokens': 19, 'total_tokens': 110, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

##### Less Manual

Sometimes LLM model may need multiple calls to the tool to get the desired result. Suppose we ask stock price of multiple stocks.
In the less manual way, we'll need to call the tool with batch and feed the result to the model and we'll do this with a ChatPromptTemplate that contains messages placeholder.

In [121]:
from langchain_core.prompts import ChatPromptTemplate

# we can define a template that can be used to contain both user input and other messages (AI, Tool)
# messages variable can be empty for the first call and later it can be filled with AI and Tool messages.
prompt_template = ChatPromptTemplate(
    [
        ("human", "{user_input}"),
        ("placeholder", "{messages}"),
    ]
)

In [122]:
# create a chain from template to LLM with tools
templated_llm_with_binded_tools = prompt_template | llm_with_binded_tools

In [123]:
user_input = "What is the price of GOOGLE and APPL stocks?"
ai_message = templated_llm_with_binded_tools.invoke(user_input)
ai_message.tool_calls


[{'name': 'get_stock_price',
  'args': {'symbol': 'GOOGL'},
  'id': 'call_yHuMIp49vAo0MFzXBl5Nq4WP',
  'type': 'tool_call'},
 {'name': 'get_stock_price',
  'args': {'symbol': 'AAPL'},
  'id': 'call_etGicZPqRMlZt4vGeCFbS7kz',
  'type': 'tool_call'}]

As seen above, we have multiple tool_call requests.

In [124]:
tool_msgs = get_stock_price.batch(ai_message.tool_calls)
tool_msgs

[ToolMessage(content='195.47500610351562', name='get_stock_price', tool_call_id='call_yHuMIp49vAo0MFzXBl5Nq4WP'),
 ToolMessage(content='238.91000366210938', name='get_stock_price', tool_call_id='call_etGicZPqRMlZt4vGeCFbS7kz')]

In [125]:
templated_llm_with_binded_tools.invoke({"user_input":user_input, "messages":[ai_message, *tool_msgs]})

AIMessage(content='The current price of Google (GOOGL) stock is approximately $195.48, and the price of Apple (AAPL) stock is approximately $238.91.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 135, 'total_tokens': 172, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'stop', 'logprobs': None}, id='run-bf2d437f-148a-44db-b96b-261895e616f4-0', usage_metadata={'input_tokens': 135, 'output_tokens': 37, 'total_tokens': 172, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

##### AgentExecutor

In [126]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    ("human", "{input}"), 
    ("placeholder", "{agent_scratchpad}"),
])

In [127]:
from langchain.agents import create_tool_calling_agent, AgentExecutor

agent = create_tool_calling_agent(llm, tools, prompt_template)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke({"input":"what's Google stock price"})



> Entering new AgentExecutor chain...

Invoking: `get_stock_price` with `{'symbol': 'GOOGL'}`


195.4199981689453The current stock price of Google (GOOGL) is approximately $195.42.

> Finished chain.


{'input': "what's Google stock price",
 'output': 'The current stock price of Google (GOOGL) is approximately $195.42.'}

##### Chain

Using chaining with @chain decator, we can even simplify the scenario in `Less Manual` section.

In [128]:
import datetime
from langchain_core.runnables import RunnableConfig, chain

# again we'll use a template that can contain messages
today = datetime.datetime.today().strftime("%D")
prompt_template = ChatPromptTemplate(
    [
        ("human", "{user_input}"),
        ("placeholder", "{messages}"),
    ]
)

In [129]:
# define the chain to combine user input, ai messages and tool messages
@chain
def tool_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = templated_llm_with_binded_tools.invoke(input_, config=config)
    tool_msgs = get_stock_price.batch(ai_msg.tool_calls, config=config)
    return templated_llm_with_binded_tools.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

In [130]:
# invoke the chain
tool_chain.invoke("What is the price of GOOGLE and APPL stocks?")

AIMessage(content='The current stock prices are as follows:\n- **GOOGLE (GOOGL)**: $195.42\n- **Apple (AAPL)**: $238.90', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 135, 'total_tokens': 172, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bd83329f63', 'finish_reason': 'stop', 'logprobs': None}, id='run-cfeaa721-2c29-4180-8da8-d390c2ebb219-0', usage_metadata={'input_tokens': 135, 'output_tokens': 37, 'total_tokens': 172, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

#### Builtin Tools

There are many builtin tools under langchain-community packpage. These tools can be used in the model call.   
https://python.langchain.com/docs/integrations/tools/

In [131]:
%pip install -q duckduckgo-search

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [132]:
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

search.invoke({'query':"Obama's first name?"})

'The White House, official residence of the president of the United States, in July 2008. The president of the United States is the head of state and head of government of the United States, [1] indirectly elected to a four-year term via the Electoral College. [2] Under the U.S. Constitution, the officeholder leads the executive branch of the federal government and is the commander-in-chief of ... The child\'s name is "Barack Hussein Obama II" and the purported document bears much of the same information as Obama\'s authentic birth certificate from his birth at the Kapiolani Maternity ... Obama\'s father, Barack Obama, Sr., was a teenage goatherd in rural Kenya, won a scholarship to study in the United States, and eventually became a senior economist in the Kenyan government.Obama\'s mother, S. Ann Dunham, grew up in Kansas, Texas, and Washington state before her family settled in Honolulu.In 1960 she and Barack Sr. met in a Russian language class at the University of Hawaii ... Born o

### Structured Outputs

#### via Tool calling

We can use tool calling feature of the models and bind a tool to get a structured output as arguments for the tool

In [133]:
from pydantic import BaseModel, Field

class ResponseFormatter(BaseModel):
    """Always use this tool to structure your response to the user."""
    sentiment: str = Field(description="Is the text is positive, neutral or negative. Only provide one of these words")
    subject: str = Field(description="What's the subject of text with one word")
    price: float = Field(description="Price of the product. Set to 0 if no price mentioned in the text")

In [134]:
llm_with_binded_structuring_tools = llm.bind_tools([ResponseFormatter])
# Invoke the model
ai_msg = llm_with_binded_structuring_tools.invoke("Costed me 50$ but this headphone sounds nothing.")

In [135]:
ai_msg.tool_calls[0]

{'name': 'ResponseFormatter',
 'args': {'sentiment': 'negative', 'subject': 'headphone', 'price': 50},
 'id': 'call_wDKmDqk2LL65evf18QSSDGIl',
 'type': 'tool_call'}

In [136]:
# cross check the output by parsing dictionary into a pydantic object
pydantic_object = ResponseFormatter.model_validate(ai_msg.tool_calls[0]["args"])
pydantic_object

ResponseFormatter(sentiment='negative', subject='headphone', price=50.0)

#### via JSON mode (model specific)

Some models support json mode to generate json output specifically for structured outputs.

In [137]:
from langchain_core.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_template("""
Return a JSON object with keys 'sentiment' and 'subject' for the user review about a purchased product:
{review}
""")

In [138]:
from langchain_openai import ChatOpenAI

llm_with_json_output = ChatOpenAI(model="gpt-4o-mini", model_kwargs={ "response_format": { "type": "json_object" } })
templated_llm_with_json_output = template | llm_with_json_output


In [139]:
ai_msg = templated_llm_with_json_output.invoke({"review":"This headphone sounds great and I'd think of buying another pair."})

In [140]:
import json
json.loads(ai_msg.content)

{'sentiment': 'positive', 'subject': 'headphone'}

#### via langchain Structured Output

In [141]:
# Bind the schema to the model
llm_with_structured_output = llm.with_structured_output(ResponseFormatter)

templated_llm_with_structured_output = template | llm_with_structured_output

# Invoke the model
structured_output = templated_llm_with_structured_output.invoke({"review":"This headphone sounds great and I'd think of buying another pair."})
# Get back the pydantic object
structured_output

ResponseFormatter(sentiment='positive', subject='headphone', price=0.0)

### Appendix

Not all LLMs may have structured output support. In such cases, we can utilize prompts, pydantic models, and tools to get structured outputs.

In [142]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI
from pydantic import BaseModel, Field, model_validator, field_validator
from typing_extensions import Self


# instantiate a model without structured output
llm = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.0)

In [143]:
class ResponseFormatter(BaseModel):
    """Always use this tool to structure your response to the user."""
    sentiment: str = Field(description="Is the text is positive, neutral or negative. Only provide one of these words")
    subject: str = Field(description="What's the subject of text with one word")
    price: float = Field(description="Price of the product. Set to 0 if no price mentioned in the text")
    num_stars: int = Field("Possible rate value of review between 1-5 where 5 is the best")

    # You can add custom validation logic easily with Pydant for fields or model as a whole
    # field, after example
    @field_validator('num_stars', mode='after')  
    @classmethod
    def after_check_num_start(cls, value: int) -> int:
        if not(1 <= value <= 5):
            raise ValueError("Badly formed num of stars")
        return value

    # model, before example
    @model_validator(mode="before")
    @classmethod
    def before_check(cls, values: dict) -> dict:
        # values contain sentinment, subject and price as a dictionary.
        # run before the model is instantiated. These are more flexible than after validators, but they also have to deal with the raw input.
        return values
    
    # model, after example
    @model_validator(mode='after')
    def after_check(self) -> Self:
        # run after Pydantic's internal validation. They are generally more type safe and thus easier to implement.
        if self.sentiment not in ['positive', 'neutral', 'negative']:
            raise ValueError('sentiment error')
        return self

In [144]:
# Set up a parser, observe the internal formatting instructions to be passed to LLM input. We'll invoke parser with the output of LLM.
parser = PydanticOutputParser(pydantic_object=ResponseFormatter)
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"description": "Always use this tool to structure your response to the user.", "properties": {"sentiment": {"description": "Is the text is positive, neutral or negative. Only provide one of these words", "title": "Sentiment", "type": "string"}, "subject": {"description": "What's the subject of text with one word", "title": "Subject", "type": "string"}, "price": {"description": "Price of the product. Set to 0 if no price mentioned in the text", "title": "Price", "type": "number"}, "num_stars": {"default": "Possible rate value of review between

In [145]:
# inject internal formatting instructions into the prompt template. 
template = PromptTemplate(
    template="Evaluate the user review.\n{format_instructions}\n{review}\n",
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
template

PromptTemplate(input_variables=['review'], input_types={}, partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"description": "Always use this tool to structure your response to the user.", "properties": {"sentiment": {"description": "Is the text is positive, neutral or negative. Only provide one of these words", "title": "Sentiment", "type": "string"}, "subject": {"description": "What\'s the subject of text with one word", "title": "Subject", "type": "string"}, "price": {"description": "Price of the product. Set to 0 if no price mentioned in t

In [146]:
# And a query intended to prompt a language model to populate the data structure.
templated_llm = template | llm
output = templated_llm.invoke("This headphone sounds great and I'd think of buying another pair.")
parser.invoke(output)

ResponseFormatter(sentiment='positive', subject='headphone', price=0.0, num_stars=5)